In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quoradata/train.csv
/kaggle/input/quoradata/test.csv
/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [4]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 6.1 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install ipywidgets --upgrade


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.8
    Uninstalling widgetsnbextension-3.6.8:
      Successfully uninstalled widgetsnbextension-3.6.8
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 

In [10]:
pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers import evaluation, util
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load the Quora question pairs dataset
df = pd.read_csv('/kaggle/input/quoradata/train.csv')

# Preprocess and select a subset
df = df[['question1', 'question2', 'is_duplicate']].dropna()

# Split into train and test sets (keeping test set constant)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create train and test examples
train_samples = [InputExample(texts=[row['question1'], row['question2']], label=row['is_duplicate']) for _, row in train_df.iterrows()]
test_samples = [(row['question1'], row['question2'], row['is_duplicate']) for _, row in test_df.iterrows()]

# DataLoader for training
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)


In [ ]:
# Model for the Bi-encoder (off-the-shelf)
bi_encoder_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Loss function options
def train_model(model, loss_fn):
    # Define loss and train the model
    train_loss = loss_fn(model=model)
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
    return model

# Cosine Similarity Loss
cosine_loss_fn = losses.CosineSimilarityLoss
cosine_model = train_model(bi_encoder_model, cosine_loss_fn)

# Contrastive Loss
contrastive_loss_fn = losses.ContrastiveLoss
contrastive_model = train_model(bi_encoder_model, contrastive_loss_fn)

# Multiple Negative Ranking Loss
mnr_loss_fn = losses.MultipleNegativesRankingLoss
mnr_model = train_model(bi_encoder_model, mnr_loss_fn)

# Cross-encoder model
cross_encoder_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
cross_encoder_loss_fn = losses.ContrastiveLoss
cross_encoder = train_model(cross_encoder_model, cross_encoder_loss_fn)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Step,Training Loss
500,0.169900
1000,0.145600
1500,0.139300
2000,0.131900
2500,0.129600
3000,0.127200
3500,0.126300
4000,0.126100
4500,0.120100
5000,0.116600


In [ ]:
# Test the models and calculate F1-Score
def evaluate_model(model, test_data):
    y_true, y_pred = [], []
    for q1, q2, label in test_data:
        emb1 = model.encode(q1)
        emb2 = model.encode(q2)
        sim = util.pytorch_cos_sim(emb1, emb2)  # Cosine similarity
        y_true.append(label)
        y_pred.append(1 if sim >= 0.5 else 0)
    return f1_score(y_true, y_pred)

In [ ]:
# Evaluate the models
f1_cosine = evaluate_model(cosine_model, test_samples)
f1_contrastive = evaluate_model(contrastive_model, test_samples)
f1_mnr = evaluate_model(mnr_model, test_samples)
f1_cross = evaluate_model(cross_encoder, test_samples)

# Print F1-scores
print(f"F1-Score for Cosine Similarity Model: {f1_cosine}")
print(f"F1-Score for Contrastive Model: {f1_contrastive}")
print(f"F1-Score for Multiple Negatives Ranking Model: {f1_mnr}")
print(f"F1-Score for Cross Encoder Model: {f1_cross}")



In [ ]:
# Compare off-the-shelf vs fine-tuned
off_shelf_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
f1_off_shelf = evaluate_model(off_shelf_model, test_samples)
print(f"F1-Score for Off-the-Shelf Model: {f1_off_shelf}")